In [1]:
using bad

In [2]:
# construct the usual prior
pnull  = .2 # response under TAU
pmcr   = pnull + .1 # minimal clinically relevant
prior1 = Beta(mean = .35, sd = .1)  # start with an subjective prior
prior2 = update(prior1, 4, 10)      # update with phase I data
prior3 = .8*prior2 + .2*Beta(1, 1)  # robustify
prior  = prior3 <= min(.7) # restrict to plausible range

mtoer = Power(prior  | pnull)
power = Power(prior >= pmcr)
ess   = SampleSize(prior)

α, β = .05, .2
problem = Problem(
        minimise(ess),
        mtoer <= α,
        power >= 1 - β
    )
design = optimise(problem)

Building ILP Problem:   6%[=>                  ]  ETA: 0:00:16
9m  building constraints for n1=7:  ...
Building ILP Problem:  10%[=>                  ]  ETA: 0:00:15
9m  building constraints for n1=8:  ...
Building ILP Problem:  26%[=====>              ]  ETA: 0:00:05
9m  building constraints for n1=13:  ...
Building ILP Problem:  35%[=======>            ]  ETA: 0:00:03
9m  building constraints for n1=16:  ...
Building ILP Problem:  42%[========>           ]  ETA: 0:00:03
9m  building constraints for n1=18:  ...
Building ILP Problem:  48%[=========>          ]  ETA: 0:00:02
9m  building constraints for n1=20:  ...
Building ILP Problem:  55%[==========>         ]  ETA: 0:00:02
9m  building constraints for n1=22:  ...
Building ILP Problem:  61%[============>       ]  ETA: 0:00:02
9m  building constraints for n1=24:  ...
Building ILP Problem:  68%[=============>      ]  ETA: 0:00:01
9m  building constraints for n1=26:  ...
Building ILP Problem:  74%[==============>     ]  ETA: 0:00:01
9m 

GLPK Simplex Optimizer, v4.64
2217 rows, 27418 columns, 156608 non-zeros
      0: obj =   0.000000000e+00 inf =   1.800e+00 (2)
    280: obj =   3.847877516e+01 inf =   4.441e-16 (0) 2
*   866: obj =   2.647540659e+01 inf =   2.220e-16 (0) 4
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.64
2217 rows, 27418 columns, 156608 non-zeros
27418 integer variables, all of which are binary
Integer optimization begins...
+   866: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 27.0873113299
Solution found by heuristic: 26.7474499262
Solution found by heuristic: 26.6802411317
+  1219: mip =   2.668024113e+01 >=     tree is empty   0.0% (0; 149)
INTEGER OPTIMAL SOLUTION FOUND


OptimalDesign{Int64,Real}<n1=19;n2:[5,7]->[17,25]>

In [3]:
# upate with new external data
prior_ = (0.8*update(prior2, 2, 10) + 0.2*Beta(1, 1)) <= 0.7
# stage one observed outcome
x1 = 6
# partial stage two outcome (x, n)
partial_stage_two = 2, 5
# copy the deisgn
design_ = deepcopy(design)
n2_original, c2_original = n2(design, x1), c2(design, x1)
# compute conditional error / power
toer = design.problem.toer.score
pow  = design.problem.power.score
α_   = toer(design, x1, partial_stage_two = partial_stage_two)
β_   = 1 - pow(design, x1, partial_stage_two = partial_stage_two)

α_, 1 - β_

(0.16423372393676816, 0.7085811866488989)

In [4]:
# compute conditional type two error of new design
function ctoer(n2_new, c2_new)
    design_.n2[x1 + 1] = n2_new
    design_.c2[x1 + 1] = c2_new
    toer(design_, x1, partial_stage_two = partial_stage_two)
end

# same for conditional power
pow = update(pow, prior_)
function cep(n2_new, c2_new)
    design_.n2[x1 + 1] = n2_new
    design_.c2[x1 + 1] = c2_new
    pow(design_, x1, partial_stage_two = partial_stage_two)
end

cep (generic function with 1 method)

In [5]:
# iterate until solution is found
function f()
    n2_new, c2_new = partial_stage_two[2], -Inf
    a, b = ctoer(n2_new, c2_new), cep(n2_new, c2_new)
    while (a > α_) | (b < 1 - β_)
        for c2_new in vcat([-Inf], 0:(n2_new - 1), [Inf])
            a, b = ctoer(n2_new, c2_new), cep(n2_new, c2_new)
            if (a <= α_) & (b >= 1 - β_)
                return n2_new, c2_new
            end
        end
        n2_new += 1
    end
end

f (generic function with 1 method)

In [6]:
n2_new, c2_new = f()
ctoer(n2_new, c2_new), cep(n2_new, c2_new)

(0.16306230424853563, 0.7273949119534779)

In [7]:
# same but with original power level
β_ = 0.2
n2_new, c2_new = f()
ctoer(n2_new, c2_new), cep(n2_new, c2_new)

(0.15076247939144405, 0.820561600895777)